# Implementación del Problema de Optimización de Carteras con Computador Cuántico Adiabático

Este cuaderno describe la implementación de un problema de optimización de carteras de inversiones utilizando un computador cuántico adiabático. El enfoque adoptado es el del "Minimum Volatility Portfolio", utilizando variables binarias para la selección de activos. El objetivo principal es minimizar el riesgo total de la cartera, modelado por la fórmula:

$$\min \sum_{i,j} x_i \cdot w_{ij} \cdot x_j$$

donde $x_i$ representa la inclusión (1) o no (0) del activo $i$ en la cartera, y $w_{ij}$ es el término de covarianza entre los activos $i$ y $j$.

Además, se establece una restricción sobre el retorno esperado de la cartera, asegurando que sea al menos igual a un umbral predefinido. Esto se modela con una variable slack y se expresa como:

$$R \leq \sum_{i} x_i \cdot r_i$$

donde $r_i$ es el retorno esperado del activo $i$.

Finalmente, como condición adicional, se determina que solo se quiere invertir en la mitad de las empresas disponibles. Esto se traduce en la restricción:

$$\sum_{i} x_i = \frac{N}{2}$$

donde $N$ es el número total de activos disponibles.



In [1]:
!pip install -r "requirements_unix.txt"

Processing ./dadk_light_3.10.tar.bz2
  Preparing metadata (setup.py) ... done
  Created wheel for dadk: filename=dadk-2023.12.10-py3-none-any.whl size=4995459 sha256=0b902ae36d618422514d4bfd7e7ac6c239d3bc44d10575df43cb1e04f80a2119
  Stored in directory: /tmp/pip-ephem-wheel-cache-1u0ngzg6/wheels/78/36/68/08dcbec0b48f137a33fc3ac474d3838f74984d2dda7e3178dd
Successfully built dadk
  Attempting uninstall: dadk
    Found existing installation: dadk 2023.12.10
    Uninstalling dadk-2023.12.10:
      Successfully uninstalled dadk-2023.12.10


In [2]:
%matplotlib widget
from IPython.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
import random
from IPython.display import display, HTML
from dadk.Optimizer import *
from dadk.SolverFactory import *
from dadk.Solution_SolutionList import *
from dadk.BinPol import *
from random import uniform
from tabulate import tabulate
from numpy import argmax
import numpy as np
from dadk.QUBOSolverCPU import *
import pandas as pd
import numpy as np

In [3]:
# Cargar los datos del dataset
acciones = pd.read_csv('Dataset_Acciones_Pequenio.csv')

# Calcular los retornos diarios y la matriz de covarianza
# La función pct_change() calcula el cambio porcentual entre filas consecutivas, días en este caso
retornos = acciones.pct_change().dropna()
cov_matrix = retornos.cov().values

# Calcular los retornos esperados como la media de los retornos diarios (para la variable slack)
retornos_esperados = retornos.mean().values

# Obtener cantidad de activos N basado en las columnas seleccionadas
n_assets = len(acciones.columns)
asset_names = acciones.columns.tolist()

# Establecer un umbral de retorno esperado mínimo para la cartera (para la variable slack)
retorno_minimo = 0.02  # 2%

# Función para calcular el valor de stop para la variable slack
def calculate_slack_stop(retornos, retorno_min_esperado):
    # Calcular la suma máxima de los retornos
    max_returns_sum = sum(sorted(retornos, reverse=True)[:len(retornos) // 2])
    # Calcular el valor de stop para S
    slack_stop = max_returns_sum - retorno_min_esperado
    return slack_stop

# Función para construir el modelo QUBO
def build_qubo(cov_matrix, n_assets, factor_penalty, retornos, retorno_min_esperado):
    var_problema = BitArrayShape('x', (n_assets,))

    # Calcular el valor de stop para la variable slack
    slack_stop = calculate_slack_stop(retornos, retorno_min_esperado)

    # Crear la variable slack
    my_var_slack = VarSlack(name='slack_variable',start=0,step=1,stop=slack_stop, slack_type=SlackType.binary)

    var_shape_set = VarShapeSet(var_problema, my_var_slack)
    BinPol.freeze_var_shape_set(var_shape_set)

    H_cuad = BinPol()

    # Construir H_cuad utilizando la matriz de covarianza
    for i in range(n_assets):
        for j in range(n_assets):
            w_ij = cov_matrix[i, j]
            H_cuad.add_term(w_ij, ('x', i), ('x', j))

    # Construir H_slack para representar la restricción del retorno mínimo esperado
    H_slack = BinPol()

    for i in range(n_assets):
        H_slack.add_term(retornos[i], ('x', i))

    H_slack.add_slack_variable('slack_variable', factor=-1)
    H_slack.add_term(-retorno_min_esperado, ())

  # Construir H_half para representar la restricción de invertir en la mitad de empresas
    H_half = BinPol()

    for i in range(n_assets):
        H_half.add_term(1, ('x', i))
    H_half.add_term(-n_assets // 2, ())
    H_half.power(2)

    # Combinar los términos para formar el QUBO
    QUBO = H_cuad + (H_half + H_slack) * factor_penalty

    return QUBO, H_half


# Coeficiente de penalización para la restricción H_half
factor_penalty = 100
QUBO, H_half = build_qubo(cov_matrix, n_assets, factor_penalty, retornos_esperados, retorno_minimo)

# Configurar y ejecutar el solucionador QUBO
solver = QUBOSolverCPU(
    number_iterations=50000,
    number_runs=10, # Número de ejecuciones en paralelo
    scaling_bit_precision=16,
    auto_tuning=AutoTuning.AUTO_SCALING_AND_SAMPLING
)

solution_list = solver.minimize(QUBO)
print(solution_list.min_solution.configuration)
print()
print(solution_list.solver_times)


********************************************************************************
Scaling qubo, temperature_start, temperature_end and offset_increase_rate
  factor:                          108.00000
********************************************************************************


********************************************************************************
Effective values (including scaling factor)
  temperature_start:               28420.000
  temperature_end:                   114.500
  offset_increase_rate:             1080.000
  duration:                            0.002 sec
********************************************************************************

[1, 0, 0, 1]

+--------------+----------------------------+----------------------------+----------------+
| time         | from                       | to                         | duration       |
|--------------+----------------------------+----------------------------+----------------|
| anneal       | 2024-06-19 15:11:2

In [4]:
# Función para preparar y presentar el resultado
def prep_result(solution_list, H_half, silent=False):
    solution = solution_list.min_solution # La mejor solución
    constraint_penalty = H_half.compute(solution.configuration)
    if not silent:
        print(f'Valor QUBO: {constraint_penalty}')
        print(solution.configuration)
    return constraint_penalty, solution

# Función para reportar la solución
def report(constraint_penalty, solution, asset_names, retorno_min_esperado, silent=False):
    if not silent:
        if constraint_penalty == 0.0: # Es cero en las soluciones válidas
            print('Portfolio elegido:')
            # Asegurar que los índices no estén fuera del rango de asset_names
            selected_assets = [asset_names[i] for i, bit in enumerate(solution.configuration[:len(asset_names)]) if bit > 0.5]
            print(selected_assets)
        else:
            print(f"No se puede dar solución para el retorno mínimo introducido de {retorno_min_esperado}")

# Función para verificar si el retorno mínimo es alcanzable
def es_factible(retornos, retorno_min_esperado):
    retorno_posible_max = sum(sorted(retornos, reverse=True)[:len(retornos) // 2])
    return retorno_posible_max >= retorno_min_esperado

# Verificar si el retorno mínimo es alcanzable antes de continuar
if not es_factible(retornos_esperados, retorno_minimo):
    print(f"No se puede dar solución para el retorno mínimo introducido de {retorno_minimo}")
else:
    # Utilizar las funciones de resultado y reporte
    constraint_penalty, solution = prep_result(solution_list, H_half)
    report(constraint_penalty, solution, asset_names, retorno_minimo)

Valor QUBO: 0
[1, 0, 0, 1]
Portfolio elegido:
['EmpresaA', 'EmpresaD']


In [5]:
# Función para calcular la media y la desviación estándar de cada empresa
def calcular_metricas(df):
    metricas = pd.DataFrame(index=df.columns, columns=['Media', '  Desviación Estándar'])
    for column in df.columns:
        metricas.loc[column, 'Media'] = df[column].mean()
        metricas.loc[column, '  Desviación Estándar'] = df[column].std()
    return metricas

# Mostrar estadísticas de cada empresa
estadisticas = calcular_metricas(acciones)
print(estadisticas)

          Media   Desviación Estándar
EmpresaA   95.9             13.963842
EmpresaB   73.9             12.077987
EmpresaC   32.9              7.489993
EmpresaD  122.1             12.031902


Tras analizar el rendimiento de las distintas empresas así como la volatilidad de los precios de sus acciones, representados por la media y la desviación estándar respectivamente, se obtiene que las mejores empresas para invertir son la EmpresaD y la EmpresaA puesto que presentan un equilibrio favorable entre el potencial de rendimiento y el nivel de riesgo asociado.

La EmpresaD, con una media de 122.1 y una desviación estándar de 12.03, ofrece el mayor rendimiento de todas las opciones evaluadas, lo que indica una fuerte capacidad de generación de valor, manteniendo al mismo tiempo una volatilidad moderada que mitiga el riesgo de inversión.

Por su parte, la EmpresaA, aunque exhibe una desviación estándar ligeramente mayor de 13.96, también muestra un rendimiento robusto con una media de 95.9. Esta combinación sugiere que, a pesar de una mayor variabilidad en el precio de sus acciones, la EmpresaA sigue siendo una opción atractiva debido a su alta capacidad de retorno. Por lo tanto, seleccionar estas dos empresas para la inversión proporciona una diversificación estratégica que busca maximizar los beneficios ajustados por riesgo, aprovechando tanto la estabilidad relativa como el alto potencial de crecimiento.

AHORA LO MISMO PERO CON UNA COMPROBACIÓN ANTES: MIRAR CUÁL ME GUSTA MÁS A LA HORA DE EJECUTARLO

In [7]:

# Cargar los datos del dataset
acciones = pd.read_csv('Dataset_Acciones_Pequenio.csv')

# Calcular los retornos diarios y la matriz de covarianza
retornos = acciones.pct_change().dropna()
cov_matrix = retornos.cov().values

# Calcular los retornos esperados como la media de los retornos diarios (para la variable slack)
retornos_esperados = retornos.mean().values

# Obtener cantidad de activos N basado en las columnas seleccionadas
n_assets = len(acciones.columns)
asset_names = acciones.columns.tolist()

# Establecer un umbral de retorno esperado mínimo para la cartera (para la variable slack)
retorno_minimo = 0.2  # 2%

# Función para calcular el valor de stop para la variable slack
def calculate_slack_stop(retornos, retorno_min_esperado):
    max_returns_sum = sum(sorted(retornos, reverse=True)[:len(retornos) // 2])
    slack_stop = max_returns_sum - retorno_min_esperado
    return slack_stop

# Función para construir el modelo QUBO
def build_qubo(cov_matrix, n_assets, factor_penalty, retornos, retorno_min_esperado):
    var_problema = BitArrayShape('x', (n_assets,))

    slack_stop = calculate_slack_stop(retornos, retorno_min_esperado)

    my_var_slack = VarSlack(name='slack_variable', start=0, step=1, stop=slack_stop, slack_type=SlackType.binary)

    var_shape_set = VarShapeSet(var_problema, my_var_slack)
    BinPol.freeze_var_shape_set(var_shape_set)

    H_cuad = BinPol()
    for i in range(n_assets):
        for j in range(n_assets):
            w_ij = cov_matrix[i, j]
            H_cuad.add_term(w_ij, ('x', i), ('x', j))

    H_slack = BinPol()
    for i in range(n_assets):
        H_slack.add_term(retornos[i], ('x', i))

    H_slack.add_slack_variable('slack_variable', factor=-1)
    H_slack.add_term(-retorno_min_esperado, ())

    H_half = BinPol()
    for i in range(n_assets):
        H_half.add_term(1, ('x', i))
    H_half.add_term(-n_assets // 2, ())
    H_half.power(2)

    QUBO = H_cuad + (H_half + H_slack) * factor_penalty

    return QUBO, H_half

# Coeficiente de penalización para la restricción H_half
factor_penalty = 100

# Verificar si el retorno mínimo es alcanzable antes de proceder
if not es_factible(retornos_esperados, retorno_minimo):
    print(f"No se puede dar solución para el retorno mínimo introducido de {retorno_minimo}")
else:
    QUBO, H_half = build_qubo(cov_matrix, n_assets, factor_penalty, retornos_esperados, retorno_minimo)

    # Configurar y ejecutar el solucionador QUBO
    solver = QUBOSolverCPU(
        number_iterations=50000,
        number_runs=10,  # Número de ejecuciones en paralelo
        scaling_bit_precision=16,
        auto_tuning=AutoTuning.AUTO_SCALING_AND_SAMPLING
    )

    solution_list = solver.minimize(QUBO)
    print(solution_list.min_solution.configuration)
    print()
    print(solution_list.solver_times)

    # Función para preparar y presentar el resultado
    def prep_result(solution_list, H_half, silent=False):
        solution = solution_list.min_solution  # La mejor solución
        constraint_penalty = H_half.compute(solution.configuration)
        if not silent:
            print(f'Valor QUBO: {constraint_penalty}')
            print(solution.configuration)
        return constraint_penalty, solution

    # Función para reportar la solución
    def report(constraint_penalty, solution, asset_names, retorno_min_esperado, silent=False):
        if not silent:
            if constraint_penalty == 0.0:  # Es cero en las soluciones válidas
                print('Portfolio elegido:')
                # Asegurar que los índices no estén fuera del rango de asset_names
                selected_assets = [asset_names[i] for i, bit in enumerate(solution.configuration[:len(asset_names)]) if bit > 0.5]
                print(selected_assets)
            else:
                print(f"No se puede dar solución para el retorno mínimo introducido de {retorno_min_esperado}")

    # Utilizar las funciones de resultado y reporte
    constraint_penalty, solution = prep_result(solution_list, H_half)
    report(constraint_penalty, solution, asset_names, retorno_minimo)

# Función para verificar si el retorno mínimo es alcanzable
def es_factible(retornos, retorno_min_esperado):
    retorno_posible_max = sum(sorted(retornos, reverse=True)[:len(retornos) // 2])
    return retorno_posible_max >= retorno_min_esperado


No se puede dar solución para el retorno mínimo introducido de 0.2


## Paso de variables binarias a enteras

A continuación se han introducirdo una serie de cambios en el código para manejar de forma más precisa la gestión de una cartera de inversiones:

1.  **Presupuesto y unidades de inversión** :

  -  *Presupuesto y Unidades de Inversión* : Se introducen variables para el presupuesto global. Se calcula el número de unidades invertibles dividiendo el presupuesto total entre la unidad de inversión y se determina el número de bits necesarios utilizando representación binaria.

  -  *Representación de variables con múltiples bits* : Para representar la cantidad invertida en cada activo se utilizan ahora múltiples bits en lugar de una única variable binaria. La varible del problema pasa ahora a definirse como una matriz de bits con dimensiones (n_assets, n_bits), donde cada fila representa un activo y cada columna un bit de la cantidad invertida.

  -  *Construcción del modelo QUBO* : Se construyen los términos H_cuad, H_slack y H_budget considerando los múltiplos de inversión y las combinaciones de bits.

  -  *Preparación y Reporte de Resultados* : Las funciones de preparación y reporte de resultados se adaptan para mostrar las cantidades invertidas por activo y evaluar la validez de las soluciones obtenidas.

  -  *Verificación de Factibilidad* : Se verifica si el retorno mínimo es alcanzable antes de proceder con la construcción del QUBO y la ejecución del solver.

In [8]:
# Calcular los retornos diarios y la matriz de covarianza
retornos = acciones.pct_change().dropna()
cov_matrix = retornos.cov().values

# Calcular los retornos esperados como la media de los retornos diarios
retornos_esperados = retornos.mean().values

# Obtener cantidad de activos N basado en las columnas seleccionadas
n_assets = len(acciones.columns)
asset_names = acciones.columns.tolist()

# Ajustar dinámicamente el umbral de retorno mínimo
retorno_minimo = 0.02 #max(0.01, retornos_esperados.min() + 0.01)  # Ajuste dinámico del umbral

# Presupuesto total disponible para inversión
presupuesto_total = 100
unidad_inversion = 10  # Múltiplo de la inversión
max_unidades = 50 // unidad_inversion  # Máximo número de unidades invertibles por activo
n_bits = int(np.ceil(np.log2(max_unidades + 1)))  # Número de bits necesarios para representar hasta max_unidades

# Función para construir el modelo QUBO
def build_qubo(cov_matrix, n_assets, factor_penalty, retornos, retorno_min_esperado, presupuesto, unidad_inversion, n_bits):
    var_problema = BitArrayShape('x', (n_assets, n_bits))

    # Crear la variable slack
    slack_stop = presupuesto // unidad_inversion
    my_var_slack = VarSlack(name='slack_variable', start=0, step=1, stop=slack_stop, slack_type=SlackType.binary)

    var_shape_set = VarShapeSet(var_problema, my_var_slack)
    BinPol.freeze_var_shape_set(var_shape_set)

    H_cuad = BinPol()

    # Construir H_cuad utilizando la matriz de covarianza
    for i in range(n_assets):
        for j in range(n_assets):
            for k in range(n_bits):
                for l in range(n_bits):
                    w_ij = cov_matrix[i, j] * (2**k) * (2**l) * (unidad_inversion ** 2) # Como los términos (2**k) * (2**l) representan las contribuciones de las diferentes
                    H_cuad.add_term(w_ij, ('x', i, k), ('x', j, l))                     # escalas, por eso se eleva al cuadrado la unidad de inversión, de este modo se mantiene
                                                                                        # la coherencia dimensional con la covarianza
    # Construir H_slack para representar la restricción del retorno mínimo esperado
    H_slack = BinPol()

    for i in range(n_assets):
        for k in range(n_bits):
            H_slack.add_term(retornos[i] * (2**k) * unidad_inversion, ('x', i, k))

    H_slack.add_slack_variable('slack_variable', factor=-1)
    H_slack.add_term(-retorno_min_esperado, ())

    # Construir H_budget para representar la restricción del presupuesto
    H_budget = BinPol()

    for i in range(n_assets):
        for k in range(n_bits):
            H_budget.add_term((2**k) * unidad_inversion, ('x', i, k))
    H_budget.add_term(-presupuesto, ())
    H_budget.power(2)

    # Combinar los términos para formar el QUBO
    QUBO = H_cuad + (H_slack + H_budget) * factor_penalty

    return QUBO, H_budget

# Coeficiente de penalización para las restricciones
factor_penalty = 1000
QUBO, H_budget = build_qubo(cov_matrix, n_assets, factor_penalty, retornos_esperados, retorno_minimo, presupuesto_total, unidad_inversion, n_bits)

# Configurar y ejecutar el solucionador QUBO
solver = QUBOSolverCPU(
    number_iterations=50000,
    number_runs=16,  # Número de ejecuciones en paralelo
    scaling_bit_precision=16,
    auto_tuning=AutoTuning.AUTO_SCALING_AND_SAMPLING
)

solution_list = solver.minimize(QUBO)
print(solution_list.min_solution.configuration)
print()
print(solution_list.solver_times)

# Función para preparar y presentar el resultado
def prep_result(solution_list, H_budget, silent=False):
    solution = solution_list.min_solution  # La mejor solución
    constraint_penalty = H_budget.compute(solution.configuration)
    if not silent:
        print(f'Valor QUBO: {constraint_penalty}')
        print(solution.configuration)
    return constraint_penalty, solution

# Función para reportar la solución
def report(constraint_penalty, solution, asset_names, retorno_min_esperado, unidad_inversion, n_bits, silent=False):
    if not silent:
        if constraint_penalty == 0.0:  # Es cero en las soluciones válidas
            print('Portfolio elegido:')
            selected_assets = []
            for i in range(len(asset_names)):
                cantidad = sum(solution.configuration[i * n_bits + k] * (2**k) for k in range(n_bits))
                if cantidad > 0:
                    selected_assets.append((asset_names[i], cantidad * unidad_inversion))
            print(selected_assets)
        else:
            print(f"No se puede dar solución para el retorno mínimo introducido de {retorno_min_esperado}")

# Función para verificar si el retorno mínimo es alcanzable
def es_factible(retornos, retorno_min_esperado, n_assets):
    retorno_posible_max = sum(sorted(retornos, reverse=True)[:n_assets])
    return retorno_posible_max >= retorno_min_esperado

# Verificar si el retorno mínimo es alcanzable antes de proceder
if not es_factible(retornos_esperados, retorno_minimo, n_assets):
    print(f"No se puede dar solución para el retorno mínimo introducido de {retorno_minimo}")
else:
    # Utilizar las funciones de resultado y reporte
    constraint_penalty, solution = prep_result(solution_list, H_budget)
    report(constraint_penalty, solution, asset_names, retorno_minimo, unidad_inversion, n_bits)


Attention: Downscaling!

********************************************************************************
Scaling qubo, temperature_start, temperature_end and offset_increase_rate
  factor:                            0.00512
********************************************************************************


********************************************************************************
Effective values (including scaling factor)
  temperature_start:                1941.000
  temperature_end:                    68.600
  offset_increase_rate:              269.000
  duration:                            0.004 sec
********************************************************************************

[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

+--------------+----------------------------+----------------------------+----------------+
| time         | from                       | to                         | duration       |
|--------------+----------------------------+----------------------